In [1]:
import os
import nest_asyncio
import chess
import chess.engine
import asyncio
import re
from database.database.engine import init_db
from constants import CONN_STRING
from database.database.ask_db import delete_all_leela_tables

from sqlalchemy.orm import Session # Ensure this is also imported
from database.database.engine import AsyncDBSession # <--- ADD THIS LINE
from database.operations.collect_data import *
from database.operations.fen import analyze_fens_from_main_fen_batch,analyze_user_games_fens
await init_db(CONN_STRING)

/home/jon/anaconda3/envs/chessism_gpu/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


Database 'chessism' already exists.
Ensuring database tables exist...
Database tables checked/created.
Asynchronous database initialization complete.


In [2]:
#await delete_all_leela_tables()

In [3]:
#n_games = 10000

In [4]:
username = 'lafareto'
n_games = 20
new_games = await get_game_links_by_username(username, n_games)
#new_games[:3]
moves_data_batch = await get_all_moves_for_links_batch(new_games)
#moves_data_batch[108885029249][0]
precessed_games_data, fens_sequence_with_counters = await generate_fens_for_single_game_moves(moves_data_batch)

In [5]:
fens_to_insert, fens_to_update = await get_new_fens(fens_sequence_with_counters)

In [ ]:
await insert_to_main_fens(fens_to_insert, fens_to_update)

In [ ]:
##insert_to_main_fens

In [ ]:
simplify = simplify_fen_and_extract_counters_for_insert

In [ ]:
moves_data_batch.keys()

In [ ]:
moves_data_batch[108885029249][0]

In [ ]:
board = chess.Board()

In [ ]:
white_move_san = moves_data_batch[108885029249][0].get('white_move')

In [ ]:
move_obj_white = board.parse_san(white_move_san)

In [ ]:
move_obj_white

In [ ]:
current_fen = board.fen()

In [ ]:
current_fen

In [ ]:
f = simplify(current_fen)

In [ ]:
{'link':315413,'n_move':1+1,'fen':f.fen}

In [ ]:
simplify_fen_and_extract_counters_for_insert('rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1')

In [ ]:
a,b = await _generate_fens_for_single_game_moves(moves_data_batch)

In [ ]:
a

In [ ]:
108826158353

In [ ]:
await open_async_request("select * from moves where moves.link = '108826158353';")

In [ ]:
one_game_moves = moves_data_batch[108826158353]

In [ ]:
e = await generate_fens_for_single_game_moves(moves_data_batch)

In [ ]:
e

In [ ]:
a = 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'.split(' ')

In [ ]:
simplified_fen = ' '.join(a[:4])

In [ ]:
simplified_fen

In [ ]:
a[4]

In [ ]:
a[5]

In [ ]:
simplify_fen_and_extract_counters_for_insert('rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1')

In [ ]:
def simplify_fen_and_extract_counters_for_insert(raw_fen: str) -> MainFenCreateData:
    """
    Simplifies a FEN string by removing move counters and fullmove number,
    and prepares data for MainFen insertion.
    The initial_counters dictionary is expected to be empty or contain default values.
    This function sets initial n_games and moves_counter to 1 for a new observation
    within a batch, which will later be aggregated by insert_fens.
    """
    parts = raw_fen.split(' ')
    # The first four parts are board, active color, castling availability, en passant target square
    simplified_fen = ' '.join(parts[:4])
    
    # These counters represent the observation from *this* game,
    # and will be aggregated with existing data in `insert_fens`.
    return MainFenCreateData(
        fen=simplified_fen,
        n_games=1,
        moves_counter=f"#{parts[4]}#{parts[5]}_"
    )


In [ ]:
type(generate_fens_for_single_game_moves)

In [ ]:
generate_fens_for_single_game_moves(moves_data_batch)

In [ ]:
for game_link, moves in moves_data_batch.items():
    if not moves:
        print(game_link)
        continue
    
    game_fens_with_counters = generate_fens_for_single_game_moves(moves)
    if game_fens_with_counters:
        all_fens_with_counters.extend(game_fens_with_counters)
    else:
        print('kasjghdklasd')

In [ ]:
logger.info(f"Retrieved moves for {len(moves_data_batch)} games.")

# 2. Generate FENs for each game in the batch
for game_link, moves in moves_data_batch.items():
    if not moves:
        logger.warning(f"Skipping FEN generation for {game_link} as no moves were found.")
        continue
    
    game_fens_with_counters = generate_fens_for_single_game_moves(moves)
    if game_fens_with_counters:
        all_fens_with_counters.extend(game_fens_with_counters)
    else:
        logger.warning(f"No valid FENs generated for game {game_link} due to invalid moves or FENs (check debug logs).")

In [ ]:
async def all_main_fens() -> str:
    """
    Fetches the count of FENs in 'main_fen' table where 'n_games' is not equal to 1,
    formatted with commas.
    """
    sql_query = "SELECT COUNT(fen) FROM main_fen;"
    result = await open_async_request(sql_query)
    count = result[0][0] if result else 0
    return f"RAW {count:,}"
async def all_fens() -> str:
    sql_query = "SELECT COUNT(fen) FROM fen;"
    result = await open_async_request(sql_query)
    count = result[0][0] if result else 0
    return f"ANALYZED {count:,}"
print(await all_main_fens())
print(await all_fens())

In [ ]:
#results = await analyze_fens_from_main_fen_batch(batch_size = 100, nodes_limit = 50000)

In [ ]:
await analyze_user_games_fens(username = 'lafareto',
                              n_games_to_process= 10,
                              batch_size_fens= 100,
                              analysis_time_limit = 3.0,
                              n_nodes_limit = 50_000)
#await analyze_user_games_fens(username = 'lafareto', batch_size = 100, nodes_limit = 50000)

In [ ]:
async def all_main_fens() -> str:
    """
    Fetches the count of FENs in 'main_fen' table where 'n_games' is not equal to 1,
    formatted with commas.
    """
    sql_query = "SELECT COUNT(fen) FROM main_fen;"
    result = await open_async_request(sql_query)
    count = result[0][0] if result else 0
    return f"RAW {count:,}"
async def all_fens() -> str:
    sql_query = "SELECT COUNT(fen) FROM fen;"
    result = await open_async_request(sql_query)
    count = result[0][0] if result else 0
    return f"ANALYZED FENS {count:,}"
async def all_processed_games() -> str:
    sql_query = "SELECT COUNT(link) FROM processed_game;"
    result = await open_async_request(sql_query)
    count = result[0][0] if result else 0
    return f"GAMES THAT WE KNOW THE FENS OF {count:,}"
print(await all_main_fens())
print(await all_fens())
print(await all_processed_games())

In [ ]:
async def all_fens() -> str:
    """
    Fetches the count of FENs in 'main_fen' table where 'n_games' is not equal to 1,
    formatted with commas.
    """
    sql_query = "SELECT COUNT(fen) FROM fen;"
    result = await open_async_request(sql_query)
    count = result[0][0] if result else 0
    return f"{count:,}"

In [ ]:
await all_fens()

In [ ]:
await all_fens()

In [ ]:
async def get_main_fen_non_single_game_count() -> str:
    """
    Fetches the count of FENs in 'main_fen' table where 'n_games' is not equal to 1,
    formatted with commas.
    """
    sql_query = "SELECT COUNT(fen) FROM main_fen WHERE n_games != 1;"
    result = await open_async_request(sql_query)
    count = result[0][0] if result else 0
    return f"{count:,}"
async def get_main_fen_non_single_game_objects() -> List[Dict[str, Any]]:
    """
    Fetches FEN objects from 'main_fen' table where 'n_games' is not equal to 1.
    Returns:
        List[Dict[str, Any]]: A list of dictionaries, where each dictionary represents
                              a FEN object with 'fen', 'n_games', and 'moves_counter'.
    """
    sql_query = "SELECT fen, n_games, moves_counter FROM main_fen WHERE n_games != 1;"
    # Use fetch_as_dict=True to get results as dictionaries directly
    result = await open_async_request(sql_query, fetch_as_dict=True)
    return result



In [ ]:
e = await get_main_fen_non_single_game_objects()

In [ ]:
e[:20]

In [ ]:
async def get_main_fen_non_single_game_objects() -> List[Dict[str, Any]]:
    """
    Fetches FEN objects from 'main_fen' table where 'n_games' is not equal to 1.
    Returns:
        List[Dict[str, Any]]: A list of dictionaries, where each dictionary represents
                              a FEN object with 'fen', 'n_games', and 'moves_counter'.
    """
    sql_query = "SELECT fen, n_games, moves_counter FROM main_fen WHERE n_games > 2;" # Changed condition
    # Use fetch_as_dict=True to get results as dictionaries directly
    result = await open_async_request(sql_query, fetch_as_dict=True)
    return result


In [ ]:
e = await get_main_fen_non_single_game_objects()

In [ ]:
e[:10]

In [ ]:
from database.database.ask_db import *
new_game_links = get_new_games_links(10)
fen_set_from_games = get_fens_from_games_optimized(new_game_links)
new_fens = get_new_fens(fen_set_from_games)
fens_to_insert = [simplify_fen_and_extract_counters_for_insert(x) for x in new_fens]

In [ ]:
insert_fens(fens_to_insert)

In [ ]:
async def update_main_fen_stats(simplified_fen: str, moves_counter_str: str) -> dict:
    """
    Checks if a simplified FEN exists in the 'main_fen' table.
    If it exists, increments its 'n_games' count.
    If it does not exist, creates a new entry with 'n_games' set to 1.
    Also updates the 'moves_counter' for the FEN.

    Args:
        simplified_fen (str): The FEN string without halfmove clock and fullmove number.
        moves_counter_str (str): A string representing the halfmove clock and fullmove number (e.g., '0#18').

    Returns:
        dict: The updated or newly created MainFen record as a dictionary.
    """
    main_fen_interface = DBInterface(MainFen)
    
    try:
        # 1. Try to read the FEN to see if it already exists
        existing_fen = main_fen_interface.read_fen(simplified_fen)

        if existing_fen:
            # 2. If FEN exists, update n_games and moves_counter
            new_n_games = existing_fen['n_games'] + 1
            update_data = {
                "n_games": new_n_games,
                "moves_counter": moves_counter_str # Always update with the latest counter if needed
            }
            updated_fen = main_fen_interface.update(simplified_fen, update_data)
            print(f"Updated FEN '{simplified_fen}': n_games={new_n_games}")
            return updated_fen
        else:
            # 3. If FEN does not exist, create a new entry
            create_data = {
                "fen": simplified_fen,
                "n_games": 1,
                "moves_counter": moves_counter_str
            }
            # Assuming MainFenCreateData is used for Pydantic validation if applicable
            # For direct DBInterface.create, a dict is sufficient.
            # If you want Pydantic validation before DB write:
            # new_main_fen_data = MainFenCreateData(**create_data).model_dump()
            new_fen = main_fen_interface.create(create_data)
            print(f"Created new FEN '{simplified_fen}': n_games=1")
            return new_fen

    except Exception as e:
        print(f"Error during MainFen upsert for FEN '{simplified_fen}': {e}")
        raise # Re-raise to propagate the error

# --- Example Usage (assuming this function is integrated and open_request is available) ---
async def demo_main_fen_update():
    print("--- Demonstrating MainFen Update Logic ---")

    # Example 1: First time a FEN appears
    simplified_fen_1 = 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq -'
    counters_1 = '0#1'
    await update_main_fen_stats(simplified_fen_1, counters_1)

    # Example 2: Same FEN appears again
    await update_main_fen_stats(simplified_fen_1, '1#2') # Counters might change, but FEN is the same

    # Example 3: A different FEN appears
    simplified_fen_2 = 'rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq -'
    counters_2 = '1#1'
    await update_main_fen_stats(simplified_fen_2, counters_2)

    # Example 4: The different FEN appears again
    await update_main_fen_stats(simplified_fen_2, '0#3')

In [ ]:
# --- Example Usage ---
# Example 1: Full FEN string
full_fen_example_1 = 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'
simplified_fen_1, counters_1 = simplify_fen_and_extract_counters(full_fen_example_1)
print(f"Original FEN: {full_fen_example_1}")
print(f"Simplified FEN: {simplified_fen_1}")
print(f"Extracted Counters: {counters_1}\n")


In [ ]:

# Example 2: FEN string with more moves
full_fen_example_2 = 'r1r3k1/pp2ppbp/2np2p1/6N1/3PP1n1/P1N4Q/1PBB1q2/R3R1K1 w - - 0 18'
simplified_fen_2, counters_2 = simplify_fen_and_extract_counters(full_fen_example_2)
print(f"Original FEN: {full_fen_example_2}")
print(f"Simplified FEN: {simplified_fen_2}")
print(f"Extracted Counters: {counters_2}\n")

# Example 3: FEN that might be missing the last two fields (as discussed previously)
# The function will still return the first 4 parts and an empty string for counters
fen_without_counters = 'r1r3k1/pp2ppbp/2np2p1/6N1/3PP1n1/P1N4Q/1PBB1q2/R3R1K1 w - -'
simplified_fen_3, counters_3 = simplify_fen_and_extract_counters(fen_without_counters)
print(f"Original FEN: {fen_without_counters}")
print(f"Simplified FEN: {simplified_fen_3}")
print(f"Extracted Counters: {counters_3}\n")

# Example 4: Malformed FEN (too short)
malformed_fen = 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w'
simplified_fen_4, counters_4 = simplify_fen_and_extract_counters(malformed_fen)
print(f"Original FEN: {malformed_fen}")
print(f"Simplified FEN: {simplified_fen_4}")
print(f"Extracted Counters: {counters_4}\n")


In [ ]:
results[0]

In [ ]:
await open_async_request("select * from fen limit(10)")

In [ ]:
open_request("select link from knownfens limit(5)")

In [ ]:
open_request("select fen from rawfen")

In [ ]:
moves = get_all_moves_for_links_batch(['108823827815'])

In [ ]:
moves

In [ ]:
game_fens = get_fens_from_games_optimized([('108823827815')])

In [ ]:
game_fens

In [ ]:
game_fens = generate_fens_for_single_game_moves(moves['108823827815'])

In [ ]:
game_fens

In [ ]:
#delete_all_leela_tables()

In [ ]:
#delete_all_leela_tables()

In [ ]:
open_request('select * from fen')

In [ ]:
get_all_tables()

In [ ]:
delete_all_leela_tables()

In [ ]:
nest_asyncio.apply()
LC0_PATH = "/home/jon/workshop/leela_zero/leela_engine_python/Lc0/lc0.exe"
lc0_directory = os.path.dirname(LC0_PATH)
LC0_WEIGHTS_FILE = "791556.pb.gz" 

In [ ]:
async def initialize_lc0_engine() -> chess.engine.UciProtocol:
    engine_uci = None
    try:
        print("...Starting Leela engine...")
        transport, engine_uci = await chess.engine.popen_uci(LC0_PATH, cwd=lc0_directory)
        await engine_uci.configure({
                                    "WeightsFile": LC0_WEIGHTS_FILE,
                                    "Backend": "cuda-fp16", 
                                    "Threads": 1,
                                    "MinibatchSize": 1024 
                                })
        print("...Leela engine ready...")
        return engine_uci
    except Exception as e:
        print(f"Error initializing Lc0 engine: {e}")
        if engine_uci:
            await engine_uci.quit()
        raise

In [ ]:
engine = await initialize_lc0_engine()

In [ ]:
async def analyse(engine: chess.engine.UciProtocol,fens: list) -> dict():
    result = {}
    for fen in fens:
        board = chess.Board(fen)
        limit = chess.engine.Limit(nodes=50000)
        info = await engine.analyse(board, limit=limit)
        result[fen] = info
        result[fen]['score'] = info["score"].white().score(mate_score=10000) / 100
        result[fen]['pv'] = [move.uci() for move in result[fen]['pv']]
    return result

In [ ]:
positions_to_analyze = [
        "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1", # Starting position
        "r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3", # Ruy Lopez opening
        "8/8/6P1/4R3/8/6k1/2r5/6K1 b - - 0 1", # Complex endgame
        # Add hundreds more FEN strings here as needed
        # "r3k2r/p1qnp1b1/2p2npp/4p3/1pP1P3/3P1N2/PP1NB1PP/R1BQ1RK1 w kq - 0 1", # Example additional FEN
        # "q3r1k1/pp3p1p/4b1p1/8/8/P3PQ2/1P3PPP/R4RK1 b - - 0 1", # Another example
    ]

In [ ]:
res = await analyse(engine, positions_to_analyze)

In [ ]:
res['rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1']

In [ ]:
import os
import nest_asyncio
import chess
import chess.engine
import asyncio

# Apply nest_asyncio for Jupyter/IPython environments
nest_asyncio.apply()

# --- Configuration ---
# Set the absolute path to your Lc0 executable within WSL
LC0_PATH = ["/home/jon/workshop/leela_zero/leela_engine_python/Lc0/lc0.exe"]

# Derive the directory from the executable path for Lc0's working directory
lc0_directory = os.path.dirname(LC0_PATH[0])

# Define the path to your Lc0 network weights file
# Assuming it's in the same directory as lc0.exe
LC0_WEIGHTS_FILE = "791556.pb.gz" 
# --- End Configuration ---


async def initialize_lc0_engine() -> chess.engine.UciProtocol:
    """
    Launches and configures the Leela Chess Zero (Lc0) engine once.

    Returns:
        An initialized Lc0 engine instance (chess.engine.UciProtocol).

    Raises:
        Exception: If the engine fails to launch or configure.
    """
    engine_uci = None
    try:
        print("Launching Lc0 engine...")
        # Launch Lc0 engine
        transport, engine_uci = await chess.engine.popen_uci(
            LC0_PATH, 
            cwd=lc0_directory
        )
        
        # Configure Lc0 options
        await engine_uci.configure({
            "WeightsFile": LC0_WEIGHTS_FILE,
            "Backend": "cuda-fp16", # Use cuda-fp16 for optimal performance on RTX 4060
            "Threads": 1,
            "MinibatchSize": 1024   # Max allowed for your Lc0 build
        })
        print("Lc0 engine successfully launched and configured.")
        return engine_uci
    except Exception as e:
        print(f"Error initializing Lc0 engine: {e}")
        # Ensure engine is quit even if configuration fails
        if engine_uci:
            await engine_uci.quit()
        raise # Re-raise the exception to propagate the error


async def analyze_single_position(engine_uci: chess.engine.UciProtocol, fen: str, nodes_limit: int = 50000) -> dict:
    """
    Analyzes a single chess position using an already initialized Leela Chess Zero (Lc0) engine
    and returns its centipawn score and principal variation.

    Args:
        engine_uci: An already initialized Lc0 engine instance.
        fen: The FEN string of the position to analyze.
        nodes_limit: The maximum number of nodes Lc0 should explore (playouts).

    Returns:
        A dictionary containing the FEN, centipawn score, and principal variation (PV).
        Includes an 'error' key if analysis fails.
    """
    board = chess.Board(fen)

    try:
        # Set the search limit (nodes in this case)
        limit = chess.engine.Limit(nodes=nodes_limit)

        # Perform the analysis using the provided engine instance
        info = await engine_uci.analyse(board, limit=limit)

        # Extract the score (centipawns from White's perspective)
        # mate_score is used to give a high/low value for checkmates
        score = info["score"].white().score(mate_score=10000) 
        
        # Extract the principal variation (PV)
        pv = [move.uci() for move in info["pv"]]

        return {"fen": fen, "score": score, "pv": pv}

    except Exception as e:
        print(f"An error occurred during Lc0 analysis for FEN {fen}: {e}")
        return {"fen": fen, "error": str(e)}


async def analyze_fens_batch(engine_uci: chess.engine.UciProtocol, fens: list[str], nodes_limit: int = 50000) -> dict:
    """
    Analyzes a list of FEN positions using a single Lc0 engine instance.

    Args:
        engine_uci: An already initialized Lc0 engine instance.
        fens: A list of FEN strings to analyze.
        nodes_limit: The maximum number of nodes Lc0 should explore for each position.

    Returns:
        A dictionary where keys are FEN strings and values are the centipawn scores (int)
        or an error string if analysis failed for that FEN.
    """
    analysis_results_dict = {}

    print("\nStarting batch analysis...")

    for i, fen in enumerate(fens):
        print(f"[{i+1}/{len(fens)}] Analyzing FEN: {fen}...")
        result = await analyze_single_position(engine_uci, fen, nodes_limit=nodes_limit) 
        
        if "error" in result:
            print(f"Failed to analyze {result['fen']}: {result['error']}")
            analysis_results_dict[result['fen']] = "Error: " + result['error']
        else:
            analysis_results_dict[result['fen']] = result['score']
            # Only print first move of PV for brevity in batch analysis progress
            print(f"  Score: {result['score']} cp, PV: {result['pv'][0]}...")
    
    return analysis_results_dict


async def main():
    """
    Main function to initialize the Lc0 engine, analyze multiple chess positions,
    and print the results.
    """
    positions_to_analyze = [
        "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1", # Starting position
        "r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3", # Ruy Lopez opening
        "8/8/6P1/4R3/8/6k1/2r5/6K1 b - - 0 1", # Complex endgame
        # Add hundreds more FEN strings here as needed
        # "r3k2r/p1qnp1b1/2p2npp/4p3/1pP1P3/3P1N2/PP1NB1PP/R1BQ1RK1 w kq - 0 1", # Example additional FEN
        # "q3r1k1/pp3p1p/4b1p1/8/8/P3PQ2/1P3PPP/R4RK1 b - - 0 1", # Another example
    ]

    # Dictionary to store final results
    final_analysis_results = {}
    engine_uci = None # Initialize engine_uci for outer finally block

    try:
        # Step 1: Initialize the Lc0 engine
        engine_uci = await initialize_lc0_engine()
        
        # Step 2: Analyze the FENs using the initialized engine
        final_analysis_results = await analyze_fens_batch(engine_uci, positions_to_analyze, nodes_limit=50000)

    except Exception as e:
        print(f"An unrecoverable error occurred in the main analysis flow: {e}")
    finally:
        # Ensure the engine is quit after all analyses are done, or if an error occurs
        if engine_uci:
            print("\nQuitting Lc0 engine...")
            await engine_uci.quit()
            print("Lc0 engine quit successfully.")

    print("\n--- Overall Analysis Complete ---")
    print("All Centipawn Scores:")
    for fen, score in final_analysis_results.items():
        print(f"FEN: {fen}")
        print(f"  Score: {score} cp")
    
    print("\nFull Results Dictionary:")
    print(final_analysis_results)


# Run the main asynchronous function
await main()


In [ ]:
async def analyze_position_with_lc0(engine_uci: chess.engine.UciProtocol, fen: str, nodes_limit: int = 50000) -> dict:
    """
    Analyzes a single chess position using an already initialized Leela Chess Zero (Lc0) engine
    and returns its centipawn score and principal variation.

    Args:
        engine_uci: An already initialized Lc0 engine instance.
        fen: The FEN string of the position to analyze.
        nodes_limit: The maximum number of nodes Lc0 should explore (playouts).

    Returns:
        A dictionary containing the FEN, centipawn score, and principal variation (PV).
        Includes an 'error' key if analysis fails.
    """
    board = chess.Board(fen)

    try:
        # Set the search limit (nodes in this case)
        limit = chess.engine.Limit(nodes=nodes_limit)

        # Perform the analysis using the provided engine instance
        info = await engine_uci.analyse(board, limit=limit)

        # Extract the score (centipawns from White's perspective)
        # mate_score is used to give a high/low value for checkmates
        score = info["score"].white().score(mate_score=10000) 
        
        # Extract the principal variation (PV)
        pv = [move.uci() for move in info["pv"]]

        return {"fen": fen, "score": score, "pv": pv}

    except Exception as e:
        print(f"An error occurred during Lc0 analysis for FEN {fen}: {e}")
        return {"fen": fen, "error": str(e)}
    # Removed the finally block from here, engine will be quit in main()

In [ ]:
import os
import nest_asyncio
import chess
import chess.engine
import asyncio

# Apply nest_asyncio for Jupyter/IPython environments
nest_asyncio.apply()

# --- Configuration ---
# Set the absolute path to your Lc0 executable within WSL
LC0_PATH = ["/home/jon/workshop/leela_zero/leela_engine_python/Lc0/lc0.exe"]

# Derive the directory from the executable path for Lc0's working directory
lc0_directory = os.path.dirname(LC0_PATH[0])

# Define the path to your Lc0 network weights file
# Assuming it's in the same directory as lc0.exe
LC0_WEIGHTS_FILE = "791556.pb.gz" 
# --- End Configuration ---


async def analyze_position_with_lc0(engine_uci: chess.engine.UciProtocol, fen: str, nodes_limit: int = 50000) -> dict:
    """
    Analyzes a single chess position using an already initialized Leela Chess Zero (Lc0) engine
    and returns its centipawn score and principal variation.

    Args:
        engine_uci: An already initialized Lc0 engine instance.
        fen: The FEN string of the position to analyze.
        nodes_limit: The maximum number of nodes Lc0 should explore (playouts).

    Returns:
        A dictionary containing the FEN, centipawn score, and principal variation (PV).
        Includes an 'error' key if analysis fails.
    """
    board = chess.Board(fen)

    try:
        # Set the search limit (nodes in this case)
        limit = chess.engine.Limit(nodes=nodes_limit)

        # Perform the analysis using the provided engine instance
        info = await engine_uci.analyse(board, limit=limit)

        # Extract the score (centipawns from White's perspective)
        # mate_score is used to give a high/low value for checkmates
        score = info["score"].white().score(mate_score=10000) 
        
        # Extract the principal variation (PV)
        pv = [move.uci() for move in info["pv"]]

        return {"fen": fen, "score": score, "pv": pv}

    except Exception as e:
        print(f"An error occurred during Lc0 analysis for FEN {fen}: {e}")
        return {"fen": fen, "error": str(e)}
    # Removed the finally block from here, engine will be quit in main()


async def main():
    """
    Main function to analyze multiple chess positions and store results.
    Initializes the Lc0 engine once and reuses it for all analyses.
    """
    positions_to_analyze = [
        "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1", # Starting position
        "r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3", # Ruy Lopez opening
        "8/8/6P1/4R3/8/6k1/2r5/6K1 b - - 0 1", # Complex endgame
        # Add hundreds more FEN strings here as needed
        # "r3k2r/p1qnp1b1/2p2npp/4p3/1pP1P3/3P1N2/PP1NB1PP/R1BQ1RK1 w kq - 0 1", # Example additional FEN
        # "q3r1k1/pp3p1p/4b1p1/8/8/P3PQ2/1P3PPP/R4RK1 b - - 0 1", # Another example
    ]

    # Dictionary to store results: {fen_string: centipawn_score}
    analysis_results_dict = {}

    print("Starting Lc0 analysis for multiple FENs...")

    engine_uci = None # Initialize engine_uci for outer finally block
    try:
        print("Launching Lc0 engine...")
        # Launch Lc0 engine once
        transport, engine_uci = await chess.engine.popen_uci(
            LC0_PATH, 
            cwd=lc0_directory
        )
        
        # Configure Lc0 options once
        await engine_uci.configure({
            "WeightsFile": LC0_WEIGHTS_FILE,
            "Backend": "cuda-fp16", # Use cuda-fp16 for optimal performance on RTX 4060
            "Threads": 1,
            "MinibatchSize": 1024   # Max allowed for your Lc0 build
        })
        print("Lc0 engine successfully launched and configured.")

        # Analyze positions one by one using the same engine instance
        for i, fen in enumerate(positions_to_analyze):
            print(f"[{i+1}/{len(positions_to_analyze)}] Analyzing FEN: {fen}...")
            # Pass the already created engine_uci object
            result = await analyze_position_with_lc0(engine_uci, fen, nodes_limit=50000) 
            
            if "error" in result:
                print(f"Failed to analyze {result['fen']}: {result['error']}")
                analysis_results_dict[result['fen']] = "Error: " + result['error']
            else:
                analysis_results_dict[result['fen']] = result['score']
                print(f"  Score: {result['score']} cp, PV: {result['pv'][0]}...")

    except Exception as e:
        print(f"An error occurred in main analysis loop: {e}")
    finally:
        # Ensure the engine is quit after all analyses are done, or if an error occurs
        if engine_uci:
            print("\nQuitting Lc0 engine...")
            await engine_uci.quit()
            print("Lc0 engine quit successfully.")

    print("\n--- Analysis Complete ---")
    print("All Centipawn Scores:")
    for fen, score in analysis_results_dict.items():
        print(f"FEN: {fen}")
        print(f"  Score: {score} cp")
    
    print("\nFull Results Dictionary:")
    print(analysis_results_dict)


# Run the main asynchronous function
await main()

In [ ]:
nest_asyncio.apply()
LC0_PATH = ["/home/jon/workshop/leela_zero/leela_engine_python/Lc0/lc0.exe"]
lc0_directory = os.path.dirname(LC0_PATH[0])
LC0_WEIGHTS_FILE = "791556.pb.gz" 

In [ ]:
fen = "8/8/6P1/4R3/8/6k1/2r5/6K1 b - - 0 1"

In [ ]:
async def get_engine( nodes_limit: int = 50000):
    

In [ ]:
import os
import nest_asyncio
import chess
import chess.engine
import asyncio

# Apply nest_asyncio for Jupyter/IPython environments
nest_asyncio.apply()

# --- Configuration ---
# Set the absolute path to your Lc0 executable within WSL
LC0_PATH = ["/home/jon/workshop/leela_zero/leela_engine_python/Lc0/lc0.exe"]

# Derive the directory from the executable path for Lc0's working directory
lc0_directory = os.path.dirname(LC0_PATH[0])

# Define the path to your Lc0 network weights file
# Assuming it's in the same directory as lc0.exe
LC0_WEIGHTS_FILE = "791556.pb.gz" 
# --- End Configuration ---


async def analyze_position_with_lc0(fen: str, nodes_limit: int = 50000) -> dict:
    """
    Analyzes a single chess position using Leela Chess Zero (Lc0)
    and returns its centipawn score and principal variation.

    Args:
        fen: The FEN string of the position to analyze.
        nodes_limit: The maximum number of nodes Lc0 should explore (playouts).

    Returns:
        A dictionary containing the FEN, centipawn score, and principal variation (PV).
        Includes an 'error' key if analysis fails.
    """
    board = chess.Board(fen)
    engine_uci = None # Initialize to None for the finally block

    try:
        # Launch Lc0 engine, specifying its working directory for weights file
        # The 'engine_uci' variable now holds the actual engine object
        transport, engine_uci = await chess.engine.popen_uci(
            LC0_PATH, 
            cwd=lc0_directory
        )
        
        # Configure Lc0 options for GPU performance
        await engine_uci.configure({
            "WeightsFile": LC0_WEIGHTS_FILE,
            "Backend": "cuda-fp16", # Use cuda-fp16 for optimal performance on RTX 4060
            "Threads": 1,
            "MinibatchSize": 1024   # Max allowed for your Lc0 build
        })

        # Set the search limit (nodes in this case)
        limit = chess.engine.Limit(nodes=nodes_limit)

        # Perform the analysis
        info = await engine_uci.analyse(board, limit=limit)

        # Extract the score (centipawns from White's perspective)
        # mate_score is used to give a high/low value for checkmates
        score = info["score"].white().score(mate_score=10000) 
        
        # Extract the principal variation (PV)
        pv = [move.uci() for move in info["pv"]]

        return {"fen": fen, "score": score, "pv": pv}

    except Exception as e:
        print(f"An error occurred during Lc0 analysis for FEN {fen}: {e}")
        return {"fen": fen, "error": str(e)}
    finally:
        # Always quit the engine process to free up resources
        if engine_uci:
            await engine_uci.quit()


async def main():
    """
    Main function to analyze multiple chess positions and store results.
    """
    positions_to_analyze = [
        "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1", # Starting position
        "r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3", # Ruy Lopez opening
        "8/8/6P1/4R3/8/6k1/2r5/6K1 b - - 0 1", # Complex endgame
        # Add hundreds more FEN strings here as needed
        # "r3k2r/p1qnp1b1/2p2npp/4p3/1pP1P3/3P1N2/PP1NB1PP/R1BQ1RK1 w kq - 0 1", # Example additional FEN
        # "q3r1k1/pp3p1p/4b1p1/8/8/P3PQ2/1P3PPP/R4RK1 b - - 0 1", # Another example
    ]

    # Dictionary to store results: {fen_string: centipawn_score}
    analysis_results_dict = {}

    print("Starting Lc0 analysis for multiple FENs...")

    # Analyze positions one by one
    for i, fen in enumerate(positions_to_analyze):
        print(f"[{i+1}/{len(positions_to_analyze)}] Analyzing FEN: {fen}...")
        result = await analyze_position_with_lc0(fen, nodes_limit=50000) 
        
        if "error" in result:
            print(f"Failed to analyze {result['fen']}: {result['error']}")
            analysis_results_dict[result['fen']] = "Error: " + result['error']
        else:
            analysis_results_dict[result['fen']] = result['score']
            print(f"  Score: {result['score']} cp, PV: {result['pv'][0]}...") # Print only first move of PV for brevity

    print("\n--- Analysis Complete ---")
    print("All Centipawn Scores:")
    for fen, score in analysis_results_dict.items():
        print(f"FEN: {fen}")
        print(f"  Score: {score} cp")
    
    print("\nFull Results Dictionary:")
    print(analysis_results_dict)
    return analysis_results_dict

# Run the main asynchronous function
await main()


In [ ]:
import os
import nest_asyncio
nest_asyncio.apply()
import chess
import chess.engine
import asyncio
import subprocess

# LC0_PATH and lc0_directory definitions (assuming they are now correctly set for WSL)
LC0_PATH = ["/home/jon/workshop/leela_zero/leela_engine_python/Lc0/lc0.exe"]
lc0_directory = os.path.dirname(LC0_PATH[0])

# Define network_files (assuming it's in the same directory as lc0.exe in WSL)
network_files = [os.path.join(lc0_directory, "791556.pb.gz")] # Adjust if your weights file is different

async def analyze_position_with_lc0(fen: str, nodes_limit: int = 10000, time_limit: float = None):
    """
    Analyzes a single chess position using Leela Chess Zero (Lc0).
    """
    board = chess.Board(fen)

    # --- DEBUGGING PRINTS ---
    print(f"DEBUG: LC0_PATH being used: {LC0_PATH}")
    print(f"DEBUG: lc0_directory being used: {lc0_directory}")
    
    executable_path_str = LC0_PATH[0]
    print(f"DEBUG: Does executable file exist at '{executable_path_str}'? {os.path.exists(executable_path_str)}")
    # --- END DEBUGGING PRINTS ---

    # --- Direct subprocess.Popen test (no shell=True) ---
    print("\n--- Starting direct subprocess.Popen test (no shell=True) ---") 
    direct_test_successful = False
    process = None # Initialize process outside try for finally block
    try:
        process = subprocess.Popen(LC0_PATH, cwd=lc0_directory, 
                                   stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                                   text=True)
        
        stdout, stderr = process.communicate(input="uci\nisready\nquit\n", timeout=10) 
        print(f"DEBUG: Direct subprocess stdout:\n{stdout}")
        print(f"DEBUG: Direct subprocess stderr:\n{stderr}")
        
        if "id name Lc0" in stdout: 
            print("DEBUG: Direct subprocess test: Lc0 responded to 'uci' command (Success!).")
            direct_test_successful = True
        else:
            print("DEBUG: Direct subprocess test: Lc0 did not respond to 'uci' command as expected.")

    except FileNotFoundError as fnfe:
        print(f"DEBUG: DIRECT SUBPROCESS POPEN FAILED with FileNotFoundError: {fnfe}")
        return {"fen": fen, "error": f"Direct subprocess test failed: {fnfe}"}
    except subprocess.TimeoutExpired:
        print("DEBUG: Direct subprocess.Popen timed out. Lc0 likely started but didn't respond to 'uci' in time.")
        if process:
            process.kill()
        return {"fen": fen, "error": "Direct subprocess test timed out."}
    except Exception as e_direct_subprocess:
        print(f"DEBUG: Error during direct subprocess.Popen test: {e_direct_subprocess}")
        return {"fen": fen, "error": f"Direct subprocess test encountered an unexpected error: {e_direct_subprocess}"}
    finally:
        if process and process.poll() is None: # Check if process is still running
            process.terminate()
            try:
                process.wait(timeout=1) # Give it a moment to terminate
            except subprocess.TimeoutExpired:
                process.kill()
        print(f"--- Direct subprocess.Popen test ended (return code: {process.returncode if process else 'N/A'}) ---\n")
    
    if not direct_test_successful:
        return {"fen": fen, "error": "Direct Lc0 subprocess test failed, cannot proceed with analysis."}


    # Proceed with chess.engine.popen_uci
    # --- KEY CHANGE HERE: Unpack the tuple returned by popen_uci ---
    transport = None # Initialize transport and engine_uci for finally block
    engine_uci = None
    try:
        transport, engine_uci = await chess.engine.popen_uci(
            LC0_PATH, 
            cwd=lc0_directory
        )
        # 'engine_uci' is the actual engine object we need to work with
        
        # Lc0 specific options
        await engine_uci.configure({ # <--- USE engine_uci here
            "WeightsFile": network_files[0] if network_files else "", 
            "Backend": "cuda-fp16", # This is often the most performant if you have a CUDA GPU
            "Threads": 1,
            "MinibatchSize": 1024
        })

        limit = chess.engine.Limit(nodes=nodes_limit)
        if time_limit:
            limit = chess.engine.Limit(time=time_limit)

        print(f"Analyzing FEN: {fen} with Lc0 (Nodes: {nodes_limit}, Time: {time_limit}s)...")
        info = await engine_uci.analyse(board, limit=limit) # <--- USE engine_uci here

        score = info["score"].white().score(mate_score=10000)
        pv = [move.uci() for move in info["pv"]]

        return {"fen": fen, "score": score, "pv": pv}

    except Exception as e:
        print(f"An error occurred during Lc0 analysis with chess.engine: {e}")
        return {"fen": fen, "error": str(e)}
    finally:
        # --- KEY CHANGE HERE: Quit the engine_uci object ---
        if engine_uci: # Ensure engine_uci was successfully created
            await engine_uci.quit() # Always quit the engine process

# The main() function and the call to await main() remain the same.

import asyncio

async def main():
    positions_to_analyze = [
        "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1", # Starting position
        "r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3", # Ruy Lopez opening
        "8/8/6P1/4R3/8/6k1/2r5/6K1 b - - 0 1", # Complex endgame
    ]

    # Analyze positions one by one
    for fen in positions_to_analyze:
        result = await analyze_position_with_lc0(fen, nodes_limit=50000) # Aim for more nodes with Lc0
        if "error" in result:
            print(f"Failed to analyze {result['fen']}: {result['error']}")
        else:
            print(f"\nFEN: {result['fen']}")
            print(f"  Score (cp from White's perspective): {result['score']}")
            print(f"  Principal Variation (PV): {result['pv']}")
            print("-" * 40)

await main()

In [ ]:
kokok

In [ ]:
uciok
readyok

DEBUG: Direct subprocess stderr:
       _
|   _ | |
|_ |_ |_| v0.31.2 built Oct 20 2024

DEBUG: Direct subprocess test: Lc0 responded to 'uci' command (Success!).
--- Direct subprocess.Popen test ended (return code: 0) ---

<UciProtocol (pid=47980)>: stderr >>        _
<UciProtocol (pid=47980)>: stderr >> |   _ | |
<UciProtocol (pid=47980)>: stderr >> |_ |_ |_| v0.31.2 built Oct 20 2024

An error occurred during Lc0 analysis with chess.engine: expected value for option 'MinibatchSize' to be at most 1024, got: 2048
Failed to analyze r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3: expected value for option 'MinibatchSize' to be at most 1024, got: 2048
DEBUG: LC0_PATH being used: ['/home/jon/workshop/leela_zero/leela_engine_python/Lc0/lc0.exe']
DEBUG: lc0_directory being used: /home/jon/workshop/leela_zero/leela_engine_python/Lc0
DEBUG: Does executable file exist at '/home/jon/workshop/leela_zero/leela_engine_python/Lc0/lc0.exe'? True

In [ ]:
import os

#LC0_PATH = r"\\wsl.localhost\Ubuntu-24.04\home\jon\workshop\leela_zero\leela_engine_python\Lc0\lc0.exe"
#LC0_PATH =r"Lc0/lc0.exe"

# Change LC0_PATH to be a list containing the executable path
LC0_PATH = [r"\\wsl.localhost\Ubuntu-24.04\home\jon\workshop\leela_zero\leela_engine_python\Lc0\lc0.exe"]

# Extract the directory from the string inside the list
lc0_directory = os.path.dirname(LC0_PATH[0]) # Use LC0_PATH[0] to access the string

# Extract the directory of lc0.exe for the cwd argument
# lc0_directory = os.path.dirname(LC0_PATH) # This line is critical!

import nest_asyncio
nest_asyncio.apply()
import chess
import chess.engine

async def analyze_position_with_lc0(fen: str, nodes_limit: int = 10000, time_limit: float = None):
    """
    Analyzes a single chess position using Leela Chess Zero (Lc0).

    Args:
        fen: The FEN string of the position to analyze.
        nodes_limit: The maximum number of nodes Lc0 should explore (playouts).
                     Lc0 typically uses 'nodes' instead of 'depth' for its search limit.
        time_limit: The maximum time in seconds Lc0 should spend on analysis.

    Returns:
        A dictionary containing the analysis results (score, principal variation).
    """
    board = chess.Board(fen)
    #engine = await chess.engine.popen_uci(LC0_PATH)
    #engine = await chess.engine.popen_uci(LC0_PATH, cwd=lc0_directory) # <--- THIS IS THE FIX
    engine = await chess.engine.popen_uci(LC0_PATH, cwd=lc0_directory) # <--- UPDATED THIS LINE
    try:
        # Lc0 specific options (these are common and good starting points)
        # Check Lc0's documentation for all available options
        await engine.configure({
            "Backend": "cuda",  # Use CUDA with half-precision for speed (requires cuDNN)
                                     # Other options: "cuda", "cpu", "opencl"
            "WeightsFile": network_files[0] if network_files else "", # Lc0 usually picks newest, but explicit is good
            "Threads": 1,            # Lc0 itself is heavily GPU-parallel; 1 thread is often fine for MCTS.
                                     # More threads might be useful if you're doing heavy CPU preprocessing
                                     # or if your Lc0 build benefits from it for specific backends.
            "MinibatchSize": 2048    # Adjust based on your GPU's VRAM. Larger can be faster.
        })

        limit = chess.engine.Limit(nodes=nodes_limit)
        if time_limit:
            limit = chess.engine.Limit(time=time_limit)

        print(f"Analyzing FEN: {fen} with Lc0 (Nodes: {nodes_limit}, Time: {time_limit}s)...")
        info = await engine.analyse(board, limit=limit)

        # Lc0 returns score as "cp" (centipawns) or "mate"
        score = info["score"].white().score(mate_score=10000) # Use a large value for mate scores
        pv = [move.uci() for move in info["pv"]]

        return {"fen": fen, "score": score, "pv": pv}

    except Exception as e:
        print(f"An error occurred during Lc0 analysis: {e}")
        return {"fen": fen, "error": str(e)}
    finally:
        await engine.quit() # Always quit the engine process

import asyncio

async def main():
    positions_to_analyze = [
        "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1", # Starting position
        "r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3", # Ruy Lopez opening
        "8/8/6P1/4R3/8/6k1/2r5/6K1 b - - 0 1", # Complex endgame
    ]

    # Analyze positions one by one
    for fen in positions_to_analyze:
        result = await analyze_position_with_lc0(fen, nodes_limit=50000) # Aim for more nodes with Lc0
        if "error" in result:
            print(f"Failed to analyze {result['fen']}: {result['error']}")
        else:
            print(f"\nFEN: {result['fen']}")
            print(f"  Score (cp from White's perspective): {result['score']}")
            print(f"  Principal Variation (PV): {result['pv']}")
            print("-" * 40)

await main()

In [ ]:

if __name__ == "__main__":
    # Ensure your NVIDIA drivers and CUDA are correctly installed
    # PyTorch can be used to quickly verify CUDA setup:
    try:
        import torch
        if torch.cuda.is_available():
            print(f"PyTorch reports CUDA is available. GPU: {torch.cuda.get_device_name(0)}")
        else:
            print("PyTorch reports CUDA is NOT available. Lc0 might run on CPU, but slower.")
    except ImportError:
        print("PyTorch not installed. Cannot verify CUDA availability via PyTorch.")

    asyncio.run(main())